In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/skincare/routine.csv
/kaggle/input/skincare/skin products.csv
/kaggle/input/skincare/skin issues.csv


In [1]:
!pip install pandas
!pip install faiss-cpu
!pip install langchain==0.3.9
!pip install langchain-community==0.3.9
!pip install langchain-text-splitters==0.3.2
!pip install langchain-huggingface==0.1.2
!pip install langchain-experimental==0.3.5rc1
!pip install fastapi uvicorn pyngrok accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 87.2 MB/s eta 0:00:00:00:0100:01
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    

In [2]:
!pip install transformers==4.52.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 104.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [3]:
# CONFIGURATION
CSV_PRODUCTS = "/kaggle/input/skincare/skin products.csv"
CSV_ROUTINES = "/kaggle/input/skincare/routine.csv"
CSV_ISSUES = "/kaggle/input/skincare/skin issues.csv"
FAISS_INDEX_PATH = "models/faiss_index"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

In [4]:
from huggingface_hub import login
login()

In [5]:
from langchain_community.document_loaders.csv_loader import CSVLoader

def load_csv_files():
    all_documents = []
    csv_files = [CSV_PRODUCTS, CSV_ROUTINES, CSV_ISSUES]
    for csv_file in csv_files:
        try:
            loader = CSVLoader(
                file_path=csv_file,
                encoding="utf-8",
                csv_args={'delimiter': ','}
            )
            documents = loader.load()
            for doc in documents:
                metadata = doc.metadata
                for key in ["RoutineID", "issue_id", "ProductID", "Unnamed: 0"]:
                    metadata.pop(key, None)

                # remove lines in the text that mention those IDs
                doc.page_content = "\n".join([
                    line for line in doc.page_content.splitlines()
                    if not any(x in line for x in ["RoutineID", "issue_id", "ProductID"])
                ])

            all_documents.extend(documents)
            print(f"Loaded {csv_file}: {len(documents)} documents")

        except Exception as e:
            print(f" Error loading {csv_file}: {e}")

    return all_documents


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = splitter.split_documents(documents)
    print(f" Split into {len(chunks)} chunks")
    return chunks

In [7]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

def create_vector_store(chunks):
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
    if os.path.exists(FAISS_INDEX_PATH):
        print(f" Loading existing FAISS index from {FAISS_INDEX_PATH}")
        vector_store = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
    else:
        print(" Creating new FAISS index...")
        vector_store = FAISS.from_documents(chunks, embeddings)
        os.makedirs(FAISS_INDEX_PATH, exist_ok=True)
        vector_store.save_local(FAISS_INDEX_PATH)
        print(f"FAISS index saved to {FAISS_INDEX_PATH}")

    return vector_store

In [8]:
!pip -q install "transformers>=4.43" "accelerate>=0.33" "bitsandbytes>=0.43" "autoawq>=0.2.7" torch --extra-index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but yo

In [9]:
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
def initialize_llm():
    model_id = "mistralai/Mistral-7B-Instruct-v0.3"

    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_cfg,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id)

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=750,
        temperature=0.7,
        return_full_text=False,
    )

    llm = HuggingFacePipeline(pipeline=pipe)
    return llm


2025-12-04 17:47:52.715353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764870472.895076      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764870472.947315      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [10]:
from langchain.prompts import PromptTemplate


def create_prompt_template():
    """Create custom prompt for skincare chatbot"""
    template = """You are an expert skincare assistant. Use the provided context to answer questions about skin issues, skincare products, and routines.

Context from knowledge base:
{context}

Chat History:
{chat_history}

Question: {question}

Provide a helpful response that includes:
1. Understanding of the skin issue
2. Recommended routine steps
3. Specific product suggestions
4. Additional tips for dealing with the issue

Answer:"""

    prompt = PromptTemplate(
        template=template,
        input_variables=["context", "chat_history", "question"]
    )
    return prompt

In [11]:
from langchain.memory import ConversationBufferMemory
try:
    from langchain.chains import ConversationalRetrievalChain
except ImportError:
    from langchain_experimental.chains import ConversationalRetrievalChain

def create_conversational_chain(vector_store, llm):
    """Create conversational retrieval chain with memory and custom prompt"""
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    prompt = create_prompt_template()

    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )

    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": prompt},
        return_source_documents=True,
        verbose=False 
    )
    return qa

In [12]:
all_docs = load_csv_files()
chunks = split_documents(all_docs)
vector_store = create_vector_store(chunks)


Loaded /kaggle/input/skincare/skin products.csv: 96 documents
Loaded /kaggle/input/skincare/routine.csv: 60 documents
Loaded /kaggle/input/skincare/skin issues.csv: 10 documents
 Split into 176 chunks


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Creating new FAISS index...
FAISS index saved to models/faiss_index


In [13]:
llm = initialize_llm()

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipykernel_47/3895157358.py:29: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [15]:
qa_chain = create_conversational_chain(vector_store, llm)

/tmp/ipykernel_47/1564664424.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
query = "I have dry skin . What routine do you suggest?"
response = qa_chain({"question": query})
answer = response["answer"].split("Answer:")[-1].strip()
print("Assistant:", answer)

In [16]:
from langchain.prompts import PromptTemplate

def general_fallback(user_question):
    """
    Provides a helpful general skincare answer, or politely refuses if question is outside dermatology.
    """

    template= """
You are a dermatologist assistant.
The user asked: "{user_question}"

---
CRITICAL INSTRUCTION:
1. If the user's question is NOT about skin care, skin health, or dermatology, you MUST respond concisely that your expertise is strictly limited to skin care/dermatology.
2. If the user's question IS about skin care or dermatology, and there is no relevant data in the knowledge base, provide this structured response:

Give:
1. A general explanation
2. Basic skincare tips
3. When to see a dermatologist
4. Helpful routine steps (morning & night)

Do NOT hallucinate product names or brands. Use generic terms like 'gentle cleanser' or 'topical retinoid'.
Keep answer short and clear.
"""

    prompt_text = template.format(user_question=user_question)

    llm = initialize_llm()
    return llm(prompt_text)


In [17]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def max_cosine_similarity(user_question, vector_store):
    """Compute max cosine similarity between question and all vectors in FAISS."""
    embedding_model = vector_store.embedding_function
    q_emb = embedding_model.embed_query(user_question)  

    # Get embeddings of all vectors in FAISS
    all_emb = vector_store.index.reconstruct_n(0, vector_store.index.ntotal)
    
    sims = cosine_similarity([q_emb], all_emb)[0]
    return np.max(sims)


In [18]:
def smart_router(user_question, vector_store, llm):
    similarity = max_cosine_similarity(user_question, vector_store)
    if similarity < 0.25:
        return "Sorry, this question is outside the scope of this chatbot."
    
    if similarity < 0.50:
        return general_fallback(user_question)

    # Use RAG
    qa_chain = create_conversational_chain(vector_store, llm)
    response = qa_chain({"question": user_question})
    answer = response["answer"].split("Answer:")[-1].strip()
    return answer


In [36]:
query1 = "how i can deal with escema"
response = smart_router(query1 , vector_store, llm)

print("Assistant:", response)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/tmp/ipykernel_47/1874292145.py:32: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm(prompt_text)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Assistant: 
---

Eczema is a skin condition that causes the skin to become itchy, red, and inflamed. To help manage eczema, it's important to:

1. Keep your skin moisturized with a fragrance-free, hypoallergenic moisturizer.
2. Avoid irritants, such as harsh soaps and detergents.
3. Use a gentle cleanser to wash your skin.
4. Apply a cool compress to affected areas for relief.
5. See a dermatologist for proper diagnosis and treatment recommendations.

For a basic skincare routine, try using:

Morning:
- Gentle cleanser
- Moisturizer
- Sunscreen (if outdoors)

Night:
- Gentle cleanser
- Moisturizer or moisturizer with steroid cream (if prescribed by a dermatologist)

Always consult with a healthcare professional for personalized advice.


In [19]:
query1 = "how many team member in football"
response = smart_router(query1 , vector_store, llm)

print("Assistant:", response)

Assistant: Sorry, this question is outside the scope of this chatbot.


In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio

In [ ]:
NGROK_TOKEN = ""
API_KEY =""

In [ ]:
from fastapi import FastAPI, Request, HTTPException

app = FastAPI()

@app.post("/generate")
async def chat(req: Request):

    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")

    data = await req.json()
    user_question = data.get("question", "").strip()

    if not user_question:
        raise HTTPException(status_code=400, detail="Missing 'question'")

    try:
        answer = smart_router(user_question, vector_store, llm)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal error: {str(e)}")

    return {"response": answer}

In [ ]:
import socket
import threading
import time
import uvicorn
from pyngrok import ngrok, conf
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)